In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("dynamic_api_call_sequence_per_malware_100_0_306.csv")

df['malware'].sum()


42797

In [5]:
features = df.iloc[:, 2:101]


,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,t_10,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
0,274,158,215,274,158,215,298,76,208,76,...,117,71,297,135,171,215,35,208,56,71
1,208,187,208,172,117,172,117,172,117,172,...,60,81,240,117,71,297,135,171,215,35
2,110,240,117,240,117,240,117,240,117,240,...,123,65,112,123,65,112,123,65,113,112
3,208,187,208,172,117,172,117,172,117,172,...,215,208,302,208,302,187,208,302,228,302
4,240,117,240,117,240,117,240,117,172,117,...,40,209,260,40,209,260,141,260,141,260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43871,240,117,240,117,240,117,240,117,172,117,...,260,141,260,141,260,141,260,141,260,141
43872,240,117,240,117,240,117,240,117,172,117,...,82,159,224,82,159,224,82,159,224,82
43873,240,117,240,117,240,117,240,117,172,117,...,141,260,141,260,141,260,141,260,141,260
43874,240,117,240,117,240,117,240,117,172,117,...,260,141,260,141,260,141,260,141,260,141


In [6]:
target = df['malware']
target

0        1
1        1
2        1
3        1
4        1
        ..
43871    1
43872    1
43873    1
43874    1
43875    1
Name: malware, Length: 43876, dtype: int64

In [ ]:
def convert_to_transition_matrix(df):
    #Identify unique API calls to facilitates the
    #initialization of the transition matrix and 
    #enables efficient mapping of API calls to indices in the matrix
    unique_api_calls = df.loc[:, 't_0':'t_99'].stack().unique()

    #Initialize transition matrix
    num_api_calls = len(unique_api_calls)
    transition_matrix = np.zeros((num_api_calls, num_api_calls))

    #Count transitions
    for _, row in df.iterrows():
        api_sequence = row['t_0':'t_99']
        for i in range(len(api_sequence) - 1):
            current_call = api_sequence[i]
            next_call = api_sequence[i + 1]
            current_index = np.where(unique_api_calls == current_call)[0][0]
            next_index = np.where(unique_api_calls == next_call)[0][0]
            transition_matrix[current_index, next_index] += 1

    #Normalize transition matrix
    transition_matrix /= transition_matrix.sum(axis=1, keepdims=True)
    return transition_matrix, unique_api_calls

In [ ]:
def convert_to_emission_matrix(dataset):
    # Get unique API calls
    unique_api_calls = dataset.loc[:, 't_0':'t_99'].stack().unique()
    num_api_calls = len(unique_api_calls)

    # Get unique class labels
    unique_classes = dataset['malware'].unique()
    num_classes = len(unique_classes)

    # Initialize emission matrix B
    B = np.zeros((num_api_calls, num_classes))

    # Count occurrences of API calls for each class
    for i, api_call in enumerate(unique_api_calls):
        for j, class_label in enumerate(unique_classes):
            # Count occurrences of API call in the class
            count = dataset[(dataset['malware'] == class_label) & (dataset[['t_0', 't_1', ..., 't_99']] == api_call).any(axis=1)].shape[0]
            B[i, j] = count

    # Normalize emission matrix B
    B /= B.sum(axis=0, keepdims=True)
    return B
